In [1]:
import flwr as fl
from flwr.server.strategy import FedAvg
from flwr.server.strategy.strategy import Strategy
from flwr.common import Context, ndarrays_to_parameters, FitIns, EvaluateIns, parameters_to_ndarrays, NDArrays
from flwr.client import NumPyClient
import torch
from torch.utils.data import DataLoader,Dataset, Subset
from torchvision.datasets import ImageFolder
from torchvision import transforms,datasets
from torchvision.models import  inception_v3
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.transforms import ToPILImage
from PIL import Image, ImageDraw
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from scipy.spatial.distance import euclidean
from collections import defaultdict, Counter
import random
import uuid
import warnings
import json
import matplotlib.pyplot as plt
import umap.umap_ as UMAP
import umap
from sklearn.decomposition import PCA
from kneed import KneeLocator
import os
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from scipy.stats import skew, kurtosis
from collections import defaultdict
import numpy as np
import math 
import cv2 ,sys
from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm
import clip as clip
import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)
from itertools import chain
import matplotlib.cm as cm 
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator


2025-08-16 22:21:20.760632: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-16 22:21:20.779304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755372080.797712 2585387 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755372080.803668 2585387 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755372080.820525 2585387 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
device = "cuda" if torch.cuda.is_available() else "CPU"
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)#ViT-B/32 ViT-L/14
target_classes = ["basketball","moving van","mountain bike","all-terrain bike","van","off-roader"]
text_tokens = clip.tokenize(target_classes).to(device)
DATASET_TO_CLIP = {
    "basketball": {"basketball"},
    "mountain bike": {"mountain bike", "all-terrain bike", "off-roader"},
    "moving van": {"moving van"},
}
with torch.no_grad():
        text_features = clip_model.encode_text(text_tokens)
        text_features /= text_features.norm(dim=-1, keepdim=True)
# === 1. Image Folder Dataset + SAM2 Segment Generation === #


#target_classes = [
#    "soccer ball",
#    "football",
#    "football match",
#    "mountain bike",
#    "racing bicycle",
#    "bike",
#    "cornears",
#    "A kernel of corn",
#    "corn on the cob",
#    "an ear of corn",
#    "corn plant",
#    "police van",
#    "lionfish",
#    "colorful lionfish",
#    "basketball",
#    "orange basketball",
#    "basketball players",
#    "basketball net",
#    "basketball clothes",
#    "van",
#    "police",
#    "truck",
#    "car",
#    "fish",
#    "lion fish",
#    "striped venomous fish",
#    "marine fish with long spiny fins",
#    "zebra-patterned reef fish",
#    "fish with feather-like fins",
#    "fish with fins"
#]

def numpy_to_native(obj):
    if isinstance(obj, dict):
        return {k: numpy_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [numpy_to_native(i) for i in obj]
    elif isinstance(obj, tuple):
        return [numpy_to_native(i) for i in obj]  # Convert tuples to lists for JSON
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    else:
        return obj
        
def get_bbox_from_mask(mask):
    ys, xs = np.where(mask)
    if len(xs) == 0 or len(ys) == 0:
        return None  # skip empty mask
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    return (x0, y0, x1, y1)
    

def save_segments_with_metadata(image_id, original_label, segments, scores,bboxes, orig_img_path,output_dir, score_threshold):
    os.makedirs(output_dir, exist_ok=True)
    metadata = []

    for i, (segment, score, bboxes) in enumerate(zip(segments, scores, bboxes)):
        if score < score_threshold:
            continue

        original_name = os.path.splitext(os.path.basename(orig_img_path))[0]
        seg_filename = f"{original_name}_{i}.png"
        seg_path = os.path.join(output_dir, seg_filename)
        Image.fromarray(segment).save(seg_path)

        metadata.append({
            "segment_id": seg_filename,
            "original_label": int(original_label),
            "original_image_index": int(image_id),
            "score": float(score),
            "bbox": bboxes,
            "original_image_path": orig_img_path
        })

    meta_path = os.path.join(output_dir, "metadata.json")
    if os.path.exists(meta_path):
        with open(meta_path, "r") as f:
            existing_metadata = json.load(f)
        existing_metadata.extend(metadata)
        metadata = existing_metadata

    with open(meta_path, "w") as f:
        json.dump(numpy_to_native(metadata), f, indent=2)

    return meta_path


# === 2. Segment Dataset with CLIP Filtering === #
def ensure_pil(img):
    if not isinstance(img, Image.Image):
        img = Image.fromarray(img)
    return img

def clip_filter_segment(seg_img_pil):
    
    img_clip = clip_preprocess(seg_img_pil).unsqueeze(0).to(device)
    with torch.no_grad():
        image_feature = clip_model.encode_image(img_clip)
        image_feature /= image_feature.norm(dim=-1, keepdim=True)
        similarity = (100.0 * image_feature @ text_features.T)
        probs = similarity.softmax(dim=-1)
        topk = 2
        top_scores, top_indices = torch.topk(probs, k=topk, dim=-1)
        #best_idx = probs.argmax(dim=-1).item()
        clip_scores = [s.item() for s in top_scores[0]]
        class_names = [target_classes[i] for i in top_indices[0].tolist()]
        # Empirical threshold to reduce false positives
        #threshold = 0.5
        return class_names, clip_scores# if clip_score >= threshold else (None, clip_score)


class SegmentDataset(Dataset):
    def __init__(self, segment_root, transform):
        self.samples = []
        self.metadata = []
        self.total_segments = 0
        self.clip_filtered_out = 0
        self.transform = transform

        for class_name in sorted(os.listdir(segment_root)):
            meta_path = os.path.join(segment_root, class_name, "metadata.json")
            if not os.path.exists(meta_path):
                print(f"[DEBUG] Metadata json missing for class: {class_name}")
                continue

            with open(meta_path, "r") as f:
                meta = json.load(f)

            for entry in meta:
                img_path = os.path.join(segment_root, class_name, entry["segment_id"])
                if not os.path.exists(img_path):
                    print(f"[DEBUG] Missing segment image file: {img_path}")
                    continue

                self.total_segments += 1

                # Do filtering at init time
                try:
                    pil_img = Image.open(img_path).convert("RGB")
                    top_classes, top_scores = clip_filter_segment(pil_img)
                    valid_clip_classes = DATASET_TO_CLIP.get(class_name, set())
                    valid_clip_classes_lower = {c.lower() for c in valid_clip_classes}
                    accepted = False
                    original_class = class_name
                    for c, s in zip(top_classes, top_scores):
                        if c.lower() in valid_clip_classes_lower and s >= 0.5:
                            clip_class = c
                            clip_score = s
                            accepted = True
                            break
                    if not accepted:
                       self.clip_filtered_out += 1
                       continue
                    if clip_class is None:
                        self.clip_filtered_out += 1
                        continue  # skip this sample
                    
                        
                    entry.update({
                        "class_name": class_name,
                        "img_path": img_path,
                        "clip_class": clip_class,
                        "clip_score": clip_score
                    })
                    self.samples.append(img_path)
                    self.metadata.append(entry)
                except Exception as e:
                    print(f"[ERROR] Failed filtering {img_path}: {e}")
                    self.clip_filtered_out += 1
                    continue

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path = self.samples[idx]
        meta = self.metadata[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(meta["original_label"])
        return img, label, meta

    def get_stats(self):
        return {
            "total_attempted": self.total_segments,
            "clip_filtered_out": self.clip_filtered_out,
            "clip_passed": len(self.samples)
        }

dino_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def custom_collate(batch):
    batch = [item for item in batch if item is not None and all(e is not None for e in item)]
    if not batch:
        return torch.empty(0), torch.empty(0), []
    images, labels, metas = zip(*batch)
    images = torch.stack(images)
    labels = torch.stack(labels)
    metas = list(metas)
    return images, labels, metas
@torch.no_grad()
def extract_features_dinov2(dataloader, model, device="cuda"):
    try:
        features, labels_all, metas_all = [], [], []
        for imgs, labels, meta_batch in tqdm(dataloader, desc="Extracting DINOv2 features"):
            if imgs.size(0) == 0:
                continue 
            imgs = imgs.to(device)
            feats = model(imgs).cpu()
            features.append(feats)
            labels_all.append(labels)
            metas_all.extend(meta_batch)
        features = torch.cat(features)
        labels_all = torch.cat(labels_all)
        return features, labels_all, metas_all
    except Exception as e:
        print(f"Client can't extract the Dino features because of : {e}")
        # Return empty or None values explicitly to avoid implicit None
        return torch.empty(0), torch.empty(0), []


In [3]:
# === 4. Clustering & Visualization  === #
def denormalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    mean = torch.tensor(mean, dtype=tensor.dtype, device=tensor.device).view(-1,1,1)
    std = torch.tensor(std, dtype=tensor.dtype, device=tensor.device).view(-1,1,1)
    return tensor * std + mean

# -- 3. Crop to mask/segment content (removes almost all background) --
def crop_to_foreground(img_np, bg_val=0, margin=4):
    # Assumes background is black (0)
    mask = np.any(img_np != bg_val, axis=-1)
    coords = np.argwhere(mask)
    if coords.size == 0:
        return img_np
    y0, x0 = np.maximum(coords.min(axis=0) - margin, 0)
    y1, x1 = np.minimum(coords.max(axis=0) + 1 + margin, img_np.shape[:2])
    return img_np[y0:y1, x0:x1]

# -- 4. Resize with preserved aspect ratio for neat thumbnails --
def resize_with_aspect(img_np, max_size=(128, 128)):
    img_pil = Image.fromarray(img_np if img_np.max() > 1 else (img_np * 255).astype('uint8'))
    img_pil = img_pil.convert("RGB")
    img_pil.thumbnail(max_size, Image.LANCZOS)
    return np.asarray(img_pil)

# -- 5. Visualization for each cluster --
def visualize_cluster_segments(cluster_meta_list, cluster_id, max_visualize=5):
    plt.figure(figsize=(15, 3))
    plt.suptitle(f"Cluster {cluster_id}, Size: {len(cluster_meta_list)}", fontsize=16, y=1.08)
    for i, meta in enumerate(cluster_meta_list[:max_visualize]):
        img_path = meta["img_path"]
        try:
            img = Image.open(img_path).convert('RGB')
            img_np = np.array(img)
            img_np_cropped = crop_to_foreground(img_np)
            img_np_display = resize_with_aspect(img_np_cropped, max_size=(128,128))
            plt.subplot(1, max_visualize, i+1)
            plt.imshow(img_np_display)
            plt.axis('off')
            plt.title(f"Cls: {meta['original_label']}\nScore: {meta['score']:.2f}\nImgID: {meta['original_image_index']}\n Clip class: {meta['clip_class']}")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            plt.subplot(1, max_visualize, i+1)
            plt.text(0.5, 0.5, 'Image Error', ha='center', va='center')
            plt.axis('off')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


In [4]:
def draw_arrow_on_image(ax, xy_from, xy_to, color='yellow', width=2, mutation_scale=15):
    """Draws an arrow from xy_from to xy_to on a matplotlib axis."""
    ax.annotate(
        '', xy=xy_to, xytext=xy_from,
        arrowprops=dict(
            arrowstyle='->',
            color=color,
            lw=width,
            shrinkA=0,
            shrinkB=0,
            mutation_scale=mutation_scale
        ),
        annotation_clip=False
    )

def create_mask_overlay_from_original(original_img_path, mask_img, alpha=0.6, mask_color=(255, 180, 0)):
    """ Overlay a binary mask onto the original full image."""
    orig = np.array(Image.open(original_img_path).convert("RGB")).copy()
    mask = np.array(mask_img)
    if mask.max() > 1:
        mask_bin = (mask[..., 0] > 0).astype(np.uint8)
    else:
        mask_bin = mask[..., 0]
    mask_rgb = np.zeros_like(orig)
    mask_rgb[mask_bin == 1] = mask_color
    out = cv2.addWeighted(mask_rgb, alpha, orig, 1 - alpha, 0)
    return Image.fromarray(out)

def visualize_clusters_with_overlay_and_arrow(clusters_by_class, max_per_cluster=3):
    """ Visualizes each cluster with segment (left) and overlay on original image (right), with arrow."""
    for class_name, cluster_info_list in clusters_by_class.items():
        n_clusters = len(cluster_info_list)
        fig, axes = plt.subplots(n_clusters, max_per_cluster, figsize=(5 * max_per_cluster, 4 * n_clusters))
        if n_clusters == 1:
            axes = np.expand_dims(axes, 0)
        plt.suptitle(f"Class: {class_name} Clusters (sorted by CLIP saliency)", fontsize=16)

        for r, (cluster, info) in enumerate(cluster_info_list):
            for c in range(max_per_cluster):
                ax = axes[r, c] if n_clusters > 1 else axes[0, c]
                try:
                    meta = cluster[c]
                except IndexError:
                    ax.axis('off')
                    continue

                # Load the segment image as saved (RGB)
                seg_img = Image.open(meta['img_path']).convert("RGB")
                seg_np = np.array(seg_img)

                # Derive binary mask (used only for overlay)
                gray = cv2.cvtColor(seg_np, cv2.COLOR_RGB2GRAY)
                mask_only = np.where(gray > 5, 255, 0).astype(np.uint8)
                mask_only = np.expand_dims(mask_only, axis=-1)

                # Load the original image and create overlay
                orig_img_path = meta['original_image_path']
                overlay_img = create_mask_overlay_from_original(orig_img_path, mask_only, alpha=0.6)

                # Resize both views
                thumb_segment = seg_img.resize((64, 64), Image.LANCZOS)
                thumb_overlay = overlay_img.resize((128, 128), Image.LANCZOS)

                # Compose side-by-side image
                combined = Image.new('RGB', (thumb_segment.width + thumb_overlay.width, thumb_overlay.height), (255, 255, 255))
                combined.paste(thumb_segment, (0, 0))
                combined.paste(thumb_overlay, (thumb_segment.width, 0))

                # Show in subplot
                ax.imshow(combined)
                ax.axis('off')

                # Draw arrow between segment and overlay
                arrow_start = (thumb_segment.width // 2, thumb_overlay.height // 2)
                arrow_end = (thumb_segment.width + thumb_overlay.width // 2, thumb_overlay.height // 2)
                draw_arrow_on_image(ax, arrow_start, arrow_end)

                # Add title with cluster info
                ax.set_title(f"CID: {info['cluster_id']} | Score: {meta['clip_score']:.2f} \n Clip Interpretation:{meta['clip_class']}", fontsize=9)

        plt.tight_layout(rect=[0, 0.03, 1, 0.98])
        plt.show()

class InceptionContext:
    def __init__(self, device):
        self.inception_model = inception_v3(weights='IMAGENET1K_V1').to(device)
        self.inception_model.eval()
        self.inception_features = {}
        self.gradients = {}
        self.inception_preproc = transforms.Compose([
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.device = device

        # Register hooks, binding self as closure via lambda or inner function
        def hook_fn(module, input, output):
            self.inception_features['mixed_8'] = output.detach()

        def save_gradient(module, grad_input, grad_output):
            self.gradients['mixed_8'] = grad_output[0]

        self.inception_model.Mixed_7b.register_forward_hook(hook_fn)
        self.inception_model.Mixed_7b.register_backward_hook(save_gradient)
def extract_mixed8_features(segment_metas, context):
    images = []
    feats = []
    for meta in segment_metas:
        img_path = meta['img_path']
        img = Image.open(img_path).convert('RGB')
        img_tensor = context.inception_preproc(img)
        images.append(img_tensor)
    if not images:
        return np.zeros((0, 1280))
    batch_size = 512
    for i in range(0, len(images), batch_size):
        batch=torch.stack(images[i:i + batch_size]).to(context.device)
        with torch.no_grad():
            _ = context.inception_model(batch)
            batch_feats = context.inception_features['mixed_8'].mean(dim=[2, 3])
        feats.append(batch_feats.cpu().numpy())    
        torch.cuda.empty_cache()
    return np.concatenate(feats, axis=0)

def get_mixed8_gradients_G(img, target_class_idx, context):
    context.inception_features.clear()
    context.gradients.clear()
    tensor = context.inception_preproc(img).unsqueeze(0).to(context.device)
    tensor.requires_grad_(True)
    output = context.inception_model(tensor)
    target_logit = output[:, target_class_idx]
    context.inception_model.zero_grad()
    target_logit.backward()
    grad = context.gradients['mixed_8'].detach().cpu().numpy().squeeze()
    grad_pooled = grad.mean(axis=(1,2))
    return grad_pooled

def compute_tcav_score_for_class_G(class_images, cav, target_class_idx, context):
    pos = 0
    for img in class_images:
        grad_pooled = get_mixed8_gradients_G(img, target_class_idx, context)
        grad_pooled = grad_pooled / np.linalg.norm(grad_pooled)
        if np.dot(grad_pooled, cav) > 0:
            pos += 1
    return pos / len(class_images)
    
#def hook_fn(module, input, output):
#        inception_features['mixed_8'] = output.detach()

#def save_gradient(module, grad_input, grad_output):
#        gradients['mixed_8'] = grad_output[0]

#        inception_model.Mixed_7b.register_forward_hook(hook_fn)
#        inception_model.Mixed_7b.register_backward_hook(save_gradient)

#        inception_preproc = transforms.Compose([
#            transforms.Resize((299, 299)),
#            transforms.ToTensor(),
#            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#        ])

#def extract_mixed8_features(segment_metas):
#            images = []
#            feats = []
#            for meta in segment_metas:
#                img_path = meta['img_path']
#                img = Image.open(img_path).convert('RGB')
#                img_tensor = inception_preproc(img)
#                images.append(img_tensor)
#            if not images:
#                return np.zeros((0, 1280))
#            batch_size = 512
#            for i in range(0, len(images), batch_size):
#                batch=torch.stack(images[i:i + batch_size]).to(self.device)
#                with torch.no_grad():
#                    _ = inception_model(batch)
#                    batch_feats = inception_features['mixed_8'].mean(dim=[2, 3])
#                feats.append(batch_feats.cpu().numpy())    
#                torch.cuda.empty_cache()
#            return np.concatenate(feats, axis=0)

def calculate_cav(concept_features, random_features):
            X = np.concatenate([concept_features, random_features], axis=0)
            y = np.array([1]*len(concept_features) + [0]*len(random_features))
            clf = LogisticRegression(max_iter=1000)
            clf.fit(X, y)
            cav = clf.coef_[0]
            cav = cav / np.linalg.norm(cav)
            mean_pos = np.mean(np.dot(concept_features, cav))
            mean_neg = np.mean(np.dot(random_features, cav))
            if mean_pos < mean_neg:
                cav = -cav
            return cav 

#def get_mixed8_gradients_G(img, target_class_idx):
#            inception_features.clear()
#            gradients.clear()
#            tensor = inception_preproc(img).unsqueeze(0).to(self.device)
#            tensor.requires_grad_(True)
#            output = inception_model(tensor)
#            target_logit = output[:, target_class_idx]
#            inception_model.zero_grad()
#            target_logit.backward()
#            grad = gradients['mixed_8'].detach().cpu().numpy().squeeze()
#            grad_pooled = grad.mean(axis=(1,2))
#            return grad_pooled

#def compute_tcav_score_for_class_G(class_images, cav, target_class_idx):
#            pos = 0
#            for img in class_images:
#                grad_pooled = get_mixed8_gradients_G(img, target_class_idx)
#                grad_pooled = grad_pooled / np.linalg.norm(grad_pooled)
#                if np.dot(grad_pooled, cav) > 0:
#                    pos += 1
#            return pos / len(class_images)

def load_test_images_by_class_G(test_dir):
            class_images = {}
            for class_label in os.listdir(test_dir):
                if class_label.startswith('.'):
                    continue
                class_path = os.path.join(test_dir, class_label)
                if not os.path.isdir(class_path):
                    continue
                imgs = []
                for fname in os.listdir(class_path):
                    if fname.lower().endswith(('png', 'jpg', 'jpeg')):
                        img = Image.open(os.path.join(class_path, fname)).convert('RGB')
                        imgs.append(img)
                class_images[class_label] = imgs
            return class_images

def plot_tcav_heatmap(tcav_matrix, class_labels, cluster_labels, client_id, out_dir="cluster_visualizations", concept_class="???"):
            """
            tcav_matrix: np.ndarray of shape (num_clusters, num_classes)
            class_labels: list of class names (columns)
            cluster_labels: list of cluster names/ids (rows)
            client_id: client numeric id
            out_dir: directory to save the output image
            """
            import matplotlib
            import matplotlib.pyplot as plt
            os.makedirs(out_dir, exist_ok=True)
            fig, ax = plt.subplots(figsize=(1.2 * len(class_labels) + 2, 0.8 * len(cluster_labels) + 3))
            cmap = matplotlib.colors.LinearSegmentedColormap.from_list('red_green', ['red', 'yellow', 'green'])
            cax = ax.imshow(tcav_matrix, cmap=cmap, vmin=0., vmax=1.)

            for i in range(tcav_matrix.shape[0]):
                for j in range(tcav_matrix.shape[1]):
                    val = tcav_matrix[i, j]
                    ax.text(j, i, f"{val:.2f}", va='center', ha='center',
                            color="black" if val < 0.5 else "white", fontsize=10, fontweight='bold')
            ax.set_xticks(np.arange(len(class_labels)))
            ax.set_yticks(np.arange(len(cluster_labels)))
            ax.set_xticklabels(class_labels, rotation=45, ha="right", fontsize=12)
            ax.set_yticklabels(cluster_labels, fontsize=12)
            ax.set_xlabel("Class label")
            ax.set_ylabel("Concept (Cluster)")
            plt.title(f"TCAV class {concept_class} Concepts Contribution\nClient {client_id}", fontsize=15, pad=25)
            plt.colorbar(cax, ax=ax, fraction=0.045)
            plt.tight_layout()
            plt.savefig(os.path.join(out_dir, f"TCAV_heatmap_class_{concept_class}_client{client_id}.png"))
            plt.close(fig)


### Federated server strategy 

In [5]:
class FedKMeansStrategy(Strategy):
    def __init__(self, initial_centroids, num_clusters=10, tol=1e-3):
        self.global_centroids = initial_centroids
        self.num_clusters = num_clusters
        self.tol = tol
        self.converged = False
        self.silhouette_history = defaultdict(lambda: defaultdict(list))
        self.global_tcav_clients = set()
        self.finished = False
        self.global_cavs = None  
        self.label_to_classname = {'basketball': 0, 'mountain bike': 1, 'moving van': 2}

    def initialize_parameters(self, client_manager):
        return ndarrays_to_parameters([])

    def configure_fit(self, server_round, parameters, client_manager):
        config = {}
        if getattr(self, "finished", False):
            print("[Server] Simulation finished after global TCAV. No more rounds.")
            return []
        print(f"[Strategy] configure_fit called for round {server_round}")
        print(f"[Strategy] Available clients: {len(client_manager.all())}")
        if len(client_manager.all()) == 0:
            return []

        if self.converged:
        # Pass flags to indicate what client should do
            #print (f" this is the global_CAV {self.global_cavs}")
            if self.global_cavs:
                print("[Server] Sending GLOBAL TCAV configs.")
                config["mode"] = "tcav"
                config["do_global_tcav"] = True  # Tell clients to run global TCAV
                config["global_cavs"] = json.dumps(self.global_cavs)
            else:
                print("[Server] Sending LOCAL TCAV configs.")
                config["mode"] = "tcav"
                config["do_global_tcav"] = False  # Only run local TCAV
        else:
            print("[Server] Still in Clustering mode ")
            config["mode"] = "cluster"
        
        #if not self.converged:
        serialized = {int(k): v.tolist() for k, v in self.global_centroids.items()}
        config["centroids"] = json.dumps(serialized) 

        dummy = ndarrays_to_parameters([np.zeros((1, 1), dtype=np.float32)])
        fit_ins = FitIns(parameters=dummy, config=config)
        return [(client, fit_ins) for client in client_manager.all().values()]

    def aggregate_fit(self, server_round, results, failures):

        if failures:
            print(f"[Server] {len(failures)} client failures in round {server_round}:")
            for failure in failures:
                print(f"  - Failure: {str(failure)}")

        if self.converged :
            dummy = ndarrays_to_parameters([np.zeros((1, 1), dtype=np.float32)])
            cavs_per_concept = defaultdict(list)  # (class_label, cluster_id) -> list of (cav, count)
            for client_proxy, fit_res in results:
                cav_results = fit_res.metrics.get("cav_results")
                if cav_results is not None:
                    cav_results = json.loads(cav_results)
                    for key, value in cav_results.items():
                        class_name, cluster_id = key.rsplit('_', 1)
                        cluster_id = int(cluster_id)
                        class_label = self.label_to_classname[class_name]
                        cav = np.array(value['cav'])
                        count = value['count']
                        cavs_per_concept[(class_label, cluster_id)].append((cav, count))
            global_cavs = {}
            for (class_label, cluster_id), cav_list in cavs_per_concept.items():
                total_count = sum(cnt for _, cnt in cav_list)
                if total_count == 0:
                    continue
                weighted_cav = sum(cav * cnt for cav, cnt in cav_list) / total_count
                weighted_cav = weighted_cav / np.linalg.norm(weighted_cav)
                global_cavs[f"{class_label}_{cluster_id}"] = weighted_cav.tolist()
            if global_cavs:  # Only set if we got CAVs!
                self.global_cavs = global_cavs
                print(f"[Server] Aggregated and set {len(global_cavs)} global CAVs for global TCAV phase.")
            for client_proxy, fit_res in results:
                    metrics = fit_res.metrics
                    if metrics.get("global_tcav_ran"):
                        self.global_tcav_clients.add(getattr(client_proxy, 'cid', str(client_proxy)))
            if len(self.global_tcav_clients) == len(results):  # or num_clients if more robust
                print("[Server] All clients finished global TCAV. Ending simulation.")
                self.finished = True
            return dummy, {}

        all_cluster_data = []
        per_class_clusters = defaultdict(list)  # class_label -> list of (centroid, count)
        for client_proxy, fit_res in results:
            #cluster_data_json = fit_res.metrics.get("cluster_data")
            client_id = getattr(client_proxy, 'cid', str(client_proxy))
            avg_silhouette_scores = fit_res.metrics.get('avg_silhouette_scores')
            if avg_silhouette_scores is not None:
                avg_silhouette_scores = json.loads(avg_silhouette_scores)
            for class_label, avg_sil in avg_silhouette_scores.items():
                self.silhouette_history[client_id][int(class_label)].append(avg_sil)
            arrays = parameters_to_ndarrays(fit_res.parameters)
            #if cluster_data_json is None:
            #    continue
            if len(arrays) != 4:
                print("Malformed client parameters")
                continue
            centroids, counts, class_labels, cluster_indices = arrays
            #cluster_data_dict = json.loads(cluster_data_json)
            for centroid, count, class_label, cl_idx in zip(centroids, counts, class_labels, cluster_indices):
                        per_class_clusters[int(class_label)].append((centroid, count, int(cl_idx)))

        new_global_centroids = {}
        for class_label, centroid_list in per_class_clusters.items():
            if not centroid_list:
                continue
            n_clusters = len(centroid_list)
            # Organize by cluster index if it is present in entry, else assign sequentially
            sum_centroids = None
            total_counts = None
            K = self.num_clusters if hasattr(self, "num_clusters") else n_clusters
            sum_centroids = np.zeros((K, centroid_list[0][0].shape[0]), dtype=np.float32)
            total_counts = np.zeros(K, dtype=np.float32)
            for i, (centroid, count,cluster_index) in enumerate(centroid_list):
                idx = i % K
                sum_centroids[idx] += centroid * count
                total_counts[idx] += count
            for idx in range(K):
                if total_counts[idx] > 0:
                    sum_centroids[idx] /= total_counts[idx]
            new_global_centroids[class_label] = sum_centroids

        max_shift = 0.0
        for class_label in new_global_centroids:
            if class_label in self.global_centroids:
                shift = np.linalg.norm(self.global_centroids[class_label] - new_global_centroids[class_label], axis=1)
                max_shift = max(max_shift, np.max(shift))
        self.global_centroids = new_global_centroids
        #shift = np.linalg.norm(self.global_centroids - new_centroids, axis=1)
        #max_shift = np.max(shift)
        print(f"[Round {server_round}] Max centroid shift: {max_shift:.2f}")
        if max_shift < self.tol:
            print("Convergence achieved. Switching to TCAV phase.")
            self.converged = True
            for client_id in self.silhouette_history:
                plt.figure(figsize=(8, 5))
                for class_label in self.silhouette_history[client_id]:
                    plt.plot(self.silhouette_history[client_id][class_label],label=f"Class {class_label}")
                plt.xlabel("Round")
                plt.ylabel("Average Silhouette Score")
                plt.title(f"Client {client_id}: Avg. Silhouette Score per Class (Clustering Rounds)")
                plt.legend()
                plt.tight_layout()
                plt.savefig(f"cluster_visualizations/silhouette_history_client{client_id}.png")
                plt.close()
        dummy = ndarrays_to_parameters([np.zeros((1, 1), dtype=np.float32)])
        return dummy, {}

    def evaluate(self, server_round, parameters):
        print(f"[Server] Global evaluation for round {server_round} skipped (no global model).")
        return 0.0, {}

    def configure_evaluate(self, server_round, parameters, client_manager):
        mode = "tcav" if self.converged else "cluster"
        config = {"mode": mode}
        serialized = {int(k): v.tolist() for k, v in self.global_centroids.items()}  # Serialize to JSON string
        config = {
        "mode": mode,
        "centroids": json.dumps(serialized)
    }
        dummy = ndarrays_to_parameters([np.zeros((1, 1), dtype=np.float32)])
        evaluate_ins = EvaluateIns(parameters=dummy, config=config)
        return [(client, evaluate_ins) for client in client_manager.all().values()]

    def aggregate_evaluate(self, server_round, results, failures):
        return 0.0, {}
        if not results:
            return 0.0, {}

        total_samples = 0
        total_inertia = 0.0
        all_cluster_sizes = defaultdict(lambda: defaultdict(int))  # class_label -> cluster_idx -> count

        for client, client_res in results:
            metrics = client_res.metrics
            num_samples = client_res.num_examples
            # Inertia is per class (sum over clusters within class)
            if "inertia" in metrics:
                class_inertia = json.loads(metrics["inertia"])
                for class_label, inertia in class_inertia.items():
                    total_inertia += float(inertia)
                    total_samples += num_samples  # You can sum per sample or per class

            if "cluster_sizes" in metrics:
                cluster_sizes = json.loads(metrics["cluster_sizes"])
                for class_label, clusters in cluster_sizes.items():
                    for cluster_idx, sz in clusters.items():
                        all_cluster_sizes[int(class_label)][int(cluster_idx)] += sz

        avg_inertia = total_inertia / total_samples if total_samples else 0.0
        print(f"\n[Server][Evaluation Round {server_round}]")
        print(f"  > Avg Inertia: {avg_inertia:.2f}")
        for class_label, c_sizes in all_cluster_sizes.items():
            print(f"  > Class {class_label} cluster sizes: {dict(c_sizes)}")
        metrics = {
            "avg_inertia": avg_inertia,
            "global_cluster_sizes": {cl: dict(c_sizes) for cl, c_sizes in all_cluster_sizes.items()}
        }
        return total_samples, metrics


### Federated Client 

In [6]:
class ThesisKMeansClient(NumPyClient):
    def __init__(self, cid, features, labels, metas, device):
        print(f"[Client] {cid} Initializing with features: {type(features)}, shape: {getattr(features, 'shape', 'no shape')}")
        print(f"[Client] {cid} Labels: {type(labels)}, length: {len(labels) if hasattr(labels, '__len__') else 'no length'}")
        print(f"[Client] {cid} Metas: {type(metas)}, length: {len(metas) if hasattr(metas, '__len__') else 'no length'}")
    
        # Validate data
        if hasattr(features, 'shape') and features.shape[0] == 0:
            raise ValueError("Features array is empty")
        if len(labels) == 0:
            raise ValueError("Labels list is empty")
        if len(metas) == 0:
            raise ValueError("Metas list is empty")
        self.features = features
        self.labels = labels
        self.metas = metas
        self.k = 15
        self.cluster_labels = None
        self.cluster_cache = None
        self.dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device).eval()
        self.device = device
        self.current_centroid = None
        self.cid = cid
        self.label_to_classname = { 0:'basketball', 1: 'mountain bike', 2: 'moving van'}
        class_feature_map = defaultdict(list)
        class_meta_map = defaultdict(list)
        self.tcav_ran =False
        self.global_cav_received = False
    def get_parameters(self, config):
        return []

            
    def fit(self, parameters, config) -> tuple[list[np.ndarray], int, dict[str, float | int | str | bytes | bool]]:
        from export_from_tcav_matrix_once import export_global_tcav_from_matrix_once
        try:
            from sklearn.metrics.pairwise import pairwise_distances_argmin
            print("📦 fit() called on client.")
            mode = config["mode"]
            
            if mode == "tcav":
                do_global_tcav = config.get("do_global_tcav", False)
                metrics = {}
                if do_global_tcav :
        #=======================>>>>>>>>>>>>>>>>  Calculating the Global Concepts<<<<<<<<<<<<===========================
                    #inception_features = {}
                    #gradients = {}
                    context = InceptionContext(self.device)
                    global_cavs = json.loads(config["global_cavs"])
                    test_dir = '/gpfs/helios/home/mahmouds/Thesis/Test'
                    class_to_inception_idx = {
                        'basketball': 816,
                        'mountain bike': 162,
                        'moving van': 191
                            }
                    test_images = load_test_images_by_class_G(test_dir) 
                    # =======================================================>>>>>>>>>>>>>>>>>>>>>>>>>>> LR-XFL <<<<<<<<<<<<<<<<<<<<<<<<<<<<<==================================
                    label_to_name = {int(cid): cname for cid, cname in self.label_to_classname.items()}
                    name_to_id = {v: k for k, v in self.label_to_classname.items()}
                    def _label_iter_only():
                        for class_name, imgs in test_images.items():
                            if class_name not in name_to_id:
                                continue
                            y = name_to_id[class_name]
                            for _ in imgs:
                                 yield  y
                    
                    # =======================================================>>>>>>>>>>>>>>>>>>>>>>>>>>> End of LR-XFL <<<<<<<<<<<<<<<<<<<<<<<<<<<<<==================================
                    Gclusters = list(global_cavs.keys())
                    print (f"listing the Global_cav keys {Gclusters}")
                    Gparsed_clusters = [tuple(map(int, key.split('_'))) for key in Gclusters]
                    print (f"listing the Gparsed_clusters {Gparsed_clusters}")
                    Gcluster_labels = [f"{self.label_to_classname.get(cl, cl)}-C{cid}" for cl, cid in Gparsed_clusters]
                    print(f"Gcluster_labels {Gcluster_labels}")
                    Gclass_labels = sorted(test_images.keys())
                    print(f"Gclass_labels {Gclass_labels}")
                    #unique_classes = sorted(set(cl for cl, _ in Gclusters))
                    tcav_matrix = np.zeros((len(Gclusters), len(Gclass_labels)))
                    for i, (class_label_int, cid) in enumerate(Gparsed_clusters):
                        class_name = self.label_to_classname[class_label_int] 
                        cav = np.array(global_cavs[f"{class_label_int}_{cid}"])
                        for j, test_class in enumerate(Gclass_labels):
                            if test_class not in test_images or class_name not in class_to_inception_idx:
                                tcav_matrix[i, j] = np.nan
                                continue
                            score = compute_tcav_score_for_class_G(test_images[test_class], cav, class_to_inception_idx[test_class], context)
                            tcav_matrix[i, j] = score
                    print (f"Global TCAV Matrix:{tcav_matrix}")
                    out_dir = "cluster_visualizations"
                    for concept_class in set(cl for cl, _ in Gparsed_clusters):
                        rows = [i for i, (cl, _) in enumerate(Gparsed_clusters) if cl == concept_class]
                        if not rows:
                            continue
                        submatrix = tcav_matrix[rows, :]
                        sub_cluster_labels = [Gcluster_labels[i] for i in rows]
                        # Save heatmap, label it as "Global"
                        plot_tcav_heatmap(submatrix, Gclass_labels, sub_cluster_labels, client_id=self.cid, out_dir=out_dir, concept_class=f"{self.label_to_classname.get(concept_class, concept_class)} (Global)")
                        print(f"[Client {self.cid}] Saved **Global** TCAV class{concept_class} concepts heatmap to {out_dir}/TCAV_heatmap_class_{concept_class}_client{self.cid}_global.png")
                    export_global_tcav_from_matrix_once(
                                                    shared_dir="./data/IMAGENET_LRXFL_SHARED",
                                                    label_to_name=label_to_name,
                                                    clusters_per_class=self.k,      # K used to build tcav_matrix
                                                    tcav_matrix=tcav_matrix,        # shape (F,C) or (C,F)
                                                    label_iter=_label_iter_only(),  # one label per image -> one row per image
                                                    threshold=0.60,                 # <-- your single cutoff for binarization
                                                    concept_names_override=None,    # or pass your own names list if you have one
                                                    overwrite=False)
                    metrics = {"global_tcav_ran": True}
                    dummy_array = np.zeros((self.k, self.features.shape[1]), dtype=np.float32)
                    return [dummy_array], 0, metrics
            #=======================>>>>>>>>>>>>>>>> End of  Calculating the Global Concepts<<<<<<<<<<<<===========================
                
                metrics["tcav_ran"] = True
                class_feature_map = defaultdict(list)
                class_meta_map = defaultdict(list)
                centroids_dict = json.loads(config["centroids"])
                centroids_dict = {int(k): np.array(v, dtype=np.float32) for k, v in centroids_dict.items()}
                print(f" [CLient {self.cid}] is Converged and start constructing the needed Clusters")
            
                for f, m in zip(self.features, self.metas):
                    class_label = m['original_label']
                    class_feature_map[class_label].append(f)
                    class_meta_map[class_label].append(m)
                #print(f"Discovered {len(self.class_feature_map)} unique classes")
                for class_label in class_feature_map:
                    if class_label not in centroids_dict:
                        print(f"Skipping class {class_label}: no converged centroids from server.")
                        continue
                    features_class = np.stack(class_feature_map[class_label])
                    metas_class =class_meta_map[class_label]
                    centroids = centroids_dict[class_label]
                    # Assign each feature to closest cluster in final centroids
                    assigned_labels = pairwise_distances_argmin(features_class, centroids)
                    for meta, cid in zip(metas_class, assigned_labels):
                        meta['cluster_id'] = int(cid)
                #self.save_state()
                print(f" [CLient {self.cid}] is Proceeding for Training CAVs and computing TCAV scores...")
                print(f" [CLient {self.cid}] Picking up the nearest 40 instaces to the clusters' centriods...")
                # === Keep only closest 40 points per cluster and update features/labels/metas =======================================
                filtered_metas_global = []                                                                                           #
                filtered_indices_global = []                                                                                         #
                for class_label in class_feature_map:                                                                                #
                    if class_label not in centroids_dict:                                                                            #
                        continue                                                                                                     #
                    features_class = np.stack(class_feature_map[class_label])                                                        #
                    metas_class = class_meta_map[class_label]                                                                        #
                    centroids = centroids_dict[class_label]                                                                          #
                    assigned_labels = pairwise_distances_argmin(features_class, centroids)                                           #
                    for cid in range(centroids.shape[0]):                                                                            #
                        idxs = np.where(assigned_labels == cid)[0]                                                                   #
                        if len(idxs) == 0:                                                                                           #
                            continue                                                                                                 #
                        cluster_feats = features_class[idxs]                                                                         #
                        dists = np.linalg.norm(cluster_feats - centroids[cid], axis=1)                                               #
                        sort_idxs = np.argsort(dists)[:40]                                                                           #
                        selected_metas = [metas_class[idxs[i]] for i in sort_idxs]                                                   #
                        selected_indices = [idxs[i] for i in sort_idxs]                                                              #
                        filtered_metas_global.extend(selected_metas)                                                                 #
                        filtered_indices_global.extend([ (class_label, idx) for idx in selected_indices ])                           #
                # Rebuild features, labels, metas: (preserving exact order)                                                          #
                all_features = []                                                                                                    #
                all_labels = []                                                                                                      #
                all_metas = []                                                                                                       #
                class_to_indices = { cl: np.array([i for i in range(len(class_feature_map[cl]))]) for cl in class_feature_map }      #
                for (class_label, idx) in filtered_indices_global:                                                                   #
                    all_features.append(class_feature_map[class_label][idx])                                                         #
                    all_labels.append(class_label)                                                                                   #
                    all_metas.append(class_meta_map[class_label][idx])                                                               #
                self.features = np.stack(all_features)                                                                               #
                self.labels = np.array(all_labels)                                                                                   #
                self.metas = all_metas                                                                                               #
                cav_results=self.train_cavs_and_tcav()
                metrics["cav_results"] = json.dumps(cav_results)
                dummy_array = np.zeros((self.k, self.features.shape[1]), dtype=np.float32)
                return [dummy_array], 0, metrics  # Return NDArrays (list of np.ndarray), int, dict
            
            if "centroids" not in config:
                raise ValueError("Missing 'centroids' in config")
            avg_silhouette_scores = {}
            centroids_dict = json.loads(config["centroids"])
            centroids_dict = {int(k): np.array(v, dtype=np.float32) for k, v in centroids_dict.items()}
            class_feature_map = defaultdict(list)
            class_meta_map = defaultdict(list)
            for f, m in zip(self.features, self.metas):
                class_label = m['original_label']
                class_feature_map[class_label].append(f)
                class_meta_map[class_label].append(m)
            print(f"Discovered {len(class_feature_map)} unique classes")
            per_class_cluster_data = {}
            for class_label in class_feature_map:
                if class_label not in centroids_dict:
                    print(f"[Client {self.cid}]: Skipping class {class_label} (no centroids from server)")
                    continue
                #print(f"== [Client {self.cid}] CUDA: allocated {torch.cuda.memory_allocated() // (1024*1024)} MB, reserved {torch.cuda.memory_reserved() // (1024*1024)} MB")
                features_class = np.stack(class_feature_map[class_label])
                #print(f"== [Client {self.cid}] CUDA: allocated {torch.cuda.memory_allocated() // (1024*1024)} MB, reserved {torch.cuda.memory_reserved() // (1024*1024)} MB")
                metas_class = class_meta_map[class_label]
                #print(f"== [Client {self.cid}] CUDA: allocated {torch.cuda.memory_allocated() // (1024*1024)} MB, reserved {torch.cuda.memory_reserved() // (1024*1024)} MB")
                k = centroids_dict[class_label].shape[0]
                centroids = centroids_dict[class_label]
                print(f"\n[Client {self.cid}]  Class {class_label}: {features_class.shape[0]} segments")
                #print(f"== [Client {self.cid}] CUDA: allocated {torch.cuda.memory_allocated() // (1024*1024)} MB, reserved {torch.cuda.memory_reserved() // (1024*1024)} MB")
                if features_class.shape[0] < k:
                    print(f"[Client {self.cid}] Skipping class {class_label} due to insufficient samples for clustering")
                    continue
                print(f"[client {self.cid}] start the Kmean-Clustering  ")
                #print(f"== [Client {self.cid}] before Kmeans CUDA: allocated {torch.cuda.memory_allocated() // (1024*1024)} MB, reserved {torch.cuda.memory_reserved() // (1024*1024)} MB")
                kmeans = KMeans(n_clusters=k, init=centroids, n_init=1, max_iter=1, algorithm="elkan", random_state=42)
                kmeans.fit(features_class)
                cluster_labels = kmeans.labels_
                centers = kmeans.cluster_centers_
                if len(np.unique(cluster_labels)) > 1:
                    avg_sil = silhouette_score(features_class, cluster_labels)
                else:
                    avg_sil = -1.0
                avg_silhouette_scores[class_label] = float(avg_sil)
                #for meta, cid in zip(metas_class, cluster_labels):
                #    meta['cluster_id'] = int(cid)                    
                print(f"== [Client {self.cid}] CUDA: After Kmeans allocated {torch.cuda.memory_allocated() // (1024*1024)} MB, reserved {torch.cuda.memory_reserved() // (1024*1024)} MB")
                print(f"[client {self.cid}] Preparing for the evaluation function")
                counts = np.bincount(cluster_labels, minlength=k).astype(float)
                cluster_list = []
                for idx in range(k):
                        cluster_list.append({
                        "centroid": centers[idx].tolist(),
                        "count": counts[idx]
                        })
                per_class_cluster_data[class_label] = cluster_list
            centroids = []
            counts = []
            class_labels = []
            cluster_indices = []
            for class_label, cluster_list in per_class_cluster_data.items():
                for cluster_idx, entry in enumerate(cluster_list):
                    centroids.append(entry["centroid"])    
                    counts.append(entry["count"])
                    class_labels.append(class_label)
                    cluster_indices.append(cluster_idx)

            centroids_array = np.array(centroids, dtype=np.float32)           # shape (N, D)
            counts_array = np.array(counts, dtype=np.float32)                 # shape (N,)
            class_labels_array = np.array(class_labels, dtype=np.int32)       # shape (N,)
            cluster_indices_array = np.array(cluster_indices, dtype=np.int32) # shape (N,) 
            print(f"[client {self.cid}] building up the metrics ") 
            print(f"== [Client {self.cid}] after building the Per_class_cluster CUDA: allocated {torch.cuda.memory_allocated() // (1024*1024)} MB, reserved {torch.cuda.memory_reserved() // (1024*1024)} MB")
            #metrics = {"cluster_data": json.dumps({int(k): v for k, v in per_class_cluster_data.items()})}
            print(f"== [Client {self.cid}] after building the metrics CUDA: allocated {torch.cuda.memory_allocated() // (1024*1024)} MB, reserved {torch.cuda.memory_reserved() // (1024*1024)} MB")
            #dummy_array = np.zeros((1, 1), dtype=np.float32)
            #self.save_state()
            
            tot_samples = sum([len(class_feature_map[c]) for c in per_class_cluster_data])
            print(f"== [Client {self.cid}] After saving the state and preparing the Eval cycle parameters CUDA: allocated {torch.cuda.memory_allocated() // (1024*1024)} MB, reserved {torch.cuda.memory_reserved() // (1024*1024)} MB")
            return [centroids_array, counts_array, class_labels_array, cluster_indices_array], tot_samples, {"avg_silhouette_scores": json.dumps(avg_silhouette_scores)}  
        except Exception as e:
            import traceback
            print("❌ ERROR in client `fit()`:")
            traceback.print_exc()
            raise e  # Let Flower fail visibly

    def evaluate(self, parameters, config):
        #self.load_state()
        print(f"[Client {self.cid}] evaluate() called.")
        # Use current centroids from latest fit (already set by previous round)
        if "centroids" not in config:
            print(f"[Client {self.cid}] No centroids provided in config for evaluation.")
            return 0.0, 0, {}  # Graceful exit with defaults

        # Parse centroids from config
        try:
            centroids_dict = json.loads(config["centroids"])
            centroids_dict = {int(k): np.array(v, dtype=np.float32) for k, v in centroids_dict.items()}

        except (json.JSONDecodeError, ValueError) as e:
            print(f"[Client {self.cid}] Error parsing centroids: {e}")
            return 0.0, 0, {}  # Handle parsing errors gracefully

        # Make sure we have features and cluster labels
        if self.features is None :
            print(f"[Client {self.cid}] No features or cluster_labels available.")
            return 0.0, 0, {}

        # Compute inertia: sum of squared distances to closest centroid for each point
        from sklearn.metrics.pairwise import pairwise_distances_argmin
        class_inertia = {}
        class_cluster_sizes = {}
        total_samples = 0
        class_feature_map = defaultdict(list)
        class_meta_map = defaultdict(list)

        for f, m in zip(self.features, self.metas):
                    class_label = m['original_label']
                    class_feature_map[class_label].append(f)
                    class_meta_map[class_label].append(m)
        for class_label in class_feature_map:
            if class_label not in centroids_dict:
                continue
            features_class = np.stack(class_feature_map[class_label])
            k = centroids_dict[class_label].shape[0]
            centers = centroids_dict[class_label]
            assignments = pairwise_distances_argmin(features_class, centers)
            # Inertia: sum of squared distances from each sample to assigned centroid
            inertia = np.sum((features_class - centers[assignments]) ** 2)
            class_inertia[class_label] = float(inertia)
            # Cluster sizes
            sizes = np.bincount(assignments, minlength=k)
            
            class_cluster_sizes[class_label] = {int(idx): int(sz) for idx, sz in enumerate(sizes)}
            total_samples += len(features_class)
        
        metrics = {
            "inertia": float(inertia),
            "cluster_sizes": json.dumps(class_cluster_sizes)
        }
        return float(sum(class_inertia.values())), total_samples, metrics


    def build_clusters_by_class(self, final_clusters,final_info):
            clusters_by_class = defaultdict(list)
            for cluster, info in zip(final_clusters, final_info):
            
                if not cluster or "original_label" not in cluster[0]:
                    print("[DEBUG] Skipping malformed or empty cluster")
                    continue

                # Dynamically get the class ID from the first meta in the cluster
                class_id = cluster[0]["original_label"]
    
                # Dynamically retrieve the class name from your mapping
                class_name = self.label_to_classname[class_id]
                cluster_id = info["cluster_id"]
                cluster_size = len(cluster)
                # Sort cluster by CLIP score, highest first
                sorted_cluster = sorted(cluster, key=lambda m: m["clip_score"], reverse=True)
                
                # Optional: skip clusters where all scores are below threshold (e.g., 0.6)
                #if all(m["clip_score"] < 0.6 for m in sorted_cluster):
                #    continue
               # print(f"[DEBUG Client {self.cid}]  Class '{class_name}' (ID {class_id}), Cluster {cluster_id}:")
               # print(f"[DEBUG Client {self.cid}]         Size: {cluster_size}, Unique images: {unique_imgs}, Mean CLIP score: {mean_clip_score:.3f}")
               # print(f"[DEBUG Client {self.cid}]         Info summary: {info}")
                # Append this sorted cluster and its info to the corresponding class grouping
                clusters_by_class[class_name].append((sorted_cluster, {"cluster_id": cluster_id}))
                print(f"Accepted cluster: {class_name} | CLIP scores: {[m['clip_score'] for m in sorted_cluster]}")

            return clusters_by_class
    
############################### Visualization Helper functions ##########################
    def plot_cluster_4x4(self, cluster, cluster_id, class_label):
        # Ensure output directory exists
        out_dir = "cluster_visualizations"
        os.makedirs(out_dir, exist_ok=True)

        # Build an informative, safe filename
        fname = f"client_{self.cid}_class_{class_label}_cluster_{cluster_id}.png"
        save_path = os.path.join(out_dir, fname)
        print(f"Saving: {fname} | Class label: {class_label}, Cluster size: {len(cluster)}")
        fig = plt.figure(figsize=(12, 12))
        plt.suptitle(f"Class {class_label} | Cluster {cluster_id} | {len(cluster)} items")
        for i, meta in enumerate(cluster[:16]):
            plt.subplot(4, 4, i + 1)
            img = Image.open(meta['img_path']).convert("RGB")
            plt.imshow(img)
            plt.axis('off')
            plt.title(
            f"ImgID: {meta['original_image_index']}\n"
            f"Score: {meta['clip_score']:.2f}\n"
            f"Clip class: {meta['clip_class']}", fontsize=8
            )
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close(fig)
    def draw_arrow_on_image(self, ax, xy_from, xy_to, color='yellow', width=2, mutation_scale=15):
        """Draws an arrow from xy_from to xy_to on a matplotlib axis."""
        ax.annotate(
            '', xy=xy_to, xytext=xy_from,
            arrowprops=dict(
                arrowstyle='->',
                color=color,
                lw=width,
                shrinkA=0,
                shrinkB=0,
                mutation_scale=mutation_scale
            ),
            annotation_clip=False
        )

    def create_mask_overlay_from_original(self, original_img_path, mask_img, alpha=0.6, mask_color=(255, 180, 0)):
        """ Overlay a binary mask onto the original full image."""
        import cv2  # Ensure cv2 is imported at top if not
        orig = np.array(Image.open(original_img_path).convert("RGB")).copy()
        mask = np.array(mask_img)
        if mask.max() > 1:
           mask_bin = (mask[..., 0] > 0).astype(np.uint8)
        else:
            mask_bin = mask[..., 0]
        mask_rgb = np.zeros_like(orig)
        mask_rgb[mask_bin == 1] = mask_color
        out = cv2.addWeighted(mask_rgb, alpha, orig, 1 - alpha, 0)
        return Image.fromarray(out)


    def visualize_clusters_with_overlay_and_arrow(self, clusters_by_class, max_per_cluster=4):
        """ Visualizes each cluster with segment (left) and overlay on original image (right), with arrow."""
        for class_name, cluster_info_list in clusters_by_class.items():
            n_clusters = len(cluster_info_list)
            fig, axes = plt.subplots(n_clusters, max_per_cluster, figsize=(5 * max_per_cluster, 4 * n_clusters))
            if n_clusters == 1:
                axes = np.expand_dims(axes, 0)
            plt.suptitle(f"Class: {class_name} Clusters (sorted by CLIP saliency)", fontsize=16)

            for r, (cluster, info) in enumerate(cluster_info_list):
                for c in range(max_per_cluster):
                    ax = axes[r, c] if n_clusters > 1 else axes[0, c]
                    try:
                        meta = cluster[c]
                    except IndexError:
                        ax.axis('off')
                        continue

                # Load the segment image as saved (RGB)
                    seg_img = Image.open(meta['img_path']).convert("RGB")
                    seg_np = np.array(seg_img)

                # Derive binary mask (used only for overlay)
                    import cv2
                    gray = cv2.cvtColor(seg_np, cv2.COLOR_RGB2GRAY)
                    mask_only = np.where(gray > 5, 255, 0).astype(np.uint8)
                    mask_only = np.expand_dims(mask_only, axis=-1)

                # Load the original image and create overlay
                    orig_img_path = meta.get('original_image_path')  # Assume this field exists in meta
                    overlay_img = self.create_mask_overlay_from_original(orig_img_path, mask_only, alpha=0.6)

                # Resize both views
                    thumb_segment = seg_img.resize((64, 64), Image.LANCZOS)
                    thumb_overlay = overlay_img.resize((128, 128), Image.LANCZOS)

                # Compose side-by-side image
                    combined = Image.new('RGB', (thumb_segment.width + thumb_overlay.width, thumb_overlay.height), (255, 255, 255))
                    combined.paste(thumb_segment, (0, 0))
                    combined.paste(thumb_overlay, (thumb_segment.width, 0))

                # Show in subplot
                    ax.imshow(combined)
                    ax.axis('off')

                # Draw arrow between segment and overlay
                    arrow_start = (thumb_segment.width // 2, thumb_overlay.height // 2)
                    arrow_end = (thumb_segment.width + thumb_overlay.width // 2, thumb_overlay.height // 2)
                    self.draw_arrow_on_image(ax, arrow_start, arrow_end)

                # Add title with cluster info
                    ax.set_title(f"CID: {info['cluster_id']} | Score: {meta['clip_score']:.2f} \n Clip Interpretation: {meta['clip_class']}", fontsize=9)

            plt.tight_layout(rect=[0, 0.03, 1, 0.98])
            plt.savefig(f"cluster_visualizations/[Client_{self.cid}]Class{class_name}_cluster_segment_overlay_images.png")
            plt.close(fig)
    def visualize_clusters_umap_3d(self, structured_clusters):
        os.makedirs("cluster_visualizations", exist_ok=True)
        for class_label, clusters in structured_clusters.items():
            all_feats = []
            all_labels = []
            for cluster, info in clusters:
                cluster_feats = [f for f in self.extract_features_dinov2_from_metas(cluster)]
                cluster_id = info['cluster_id']
                all_feats.extend(cluster_feats)
                all_labels.extend([cluster_id] * len(cluster_feats))

            if len(set(all_labels)) < 2:
                continue

            reducer = umap.UMAP(n_components=3, random_state=42)
            reduced = reducer.fit_transform(np.array(all_feats))
   
            all_labels = np.array(all_labels)
            fig = plt.figure(figsize=(10, 8))
            ax = fig.add_subplot(111, projection='3d')
            colors = cm.get_cmap('tab10', np.max(all_labels)+1)
            for cluster_id in np.unique(all_labels):
                idx = all_labels == cluster_id
                ax.scatter(reduced[idx, 0], reduced[idx, 1], reduced[idx, 2],
                           label=f"Cluster {cluster_id}", alpha=0.6, s=30,
                           color=colors(cluster_id))
            ax.set_title(f"[Client {self.cid}] 3D UMAP of Class '{class_label}' by Cluster")
            ax.legend()
            ax.view_init(elev=20, azim=135)
            plt.tight_layout()
            plt.savefig(f"cluster_visualizations/[Client]_{self.cid}UMAP3D_{class_label}.png")
            plt.close(fig)

    def extract_features_dinov2_from_metas(self, metas, batch_size=256):
        img_tensors = []
        feats_accum = []
        for i, meta in enumerate(metas):
            img = Image.open(meta['img_path']).convert("RGB")
            img_tensor = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])(img)
            img_tensors.append(img_tensor)
            if len(img_tensors) == batch_size or (i == len(metas) - 1):
                batch = torch.stack(img_tensors).to(self.device)
                with torch.no_grad():
                        feats = self.dinov2(batch).cpu().numpy()
                feats_accum.extend(feats)
                del batch
                torch.cuda.empty_cache()
                img_tensors = []
        return feats_accum        
        #if not img_tensors:
#########################################################################################
    def train_cavs_and_tcav(self):
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        inception_model = inception_v3(weights='IMAGENET1K_V1').to(device)
        inception_model.eval()

        inception_features = {}
        gradients = {}
        metrics = {}
        def hook_fn(module, input, output):
            inception_features['mixed_8'] = output.detach()

        def save_gradient(module, grad_input, grad_output):
            gradients['mixed_8'] = grad_output[0]

        inception_model.Mixed_7b.register_forward_hook(hook_fn)
        inception_model.Mixed_7b.register_backward_hook(save_gradient)

        inception_preproc = transforms.Compose([
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        def extract_mixed8_features(segment_metas):
            images = []
            feats = []
            for meta in segment_metas:
                img_path = meta['img_path']
                img = Image.open(img_path).convert('RGB')
                img_tensor = inception_preproc(img)
                images.append(img_tensor)
            if not images:
                return np.zeros((0, 1280))
            batch_size = 512
            for i in range(0, len(images), batch_size):
                batch=torch.stack(images[i:i + batch_size]).to(self.device)
                with torch.no_grad():
                    _ = inception_model(batch)
                    batch_feats = inception_features['mixed_8'].mean(dim=[2, 3])
                feats.append(batch_feats.cpu().numpy())    
                torch.cuda.empty_cache()
            return np.concatenate(feats, axis=0)

        def calculate_cav(concept_features, random_features):
            X = np.concatenate([concept_features, random_features], axis=0)
            y = np.array([1]*len(concept_features) + [0]*len(random_features))
            clf = LogisticRegression(max_iter=1000)
            clf.fit(X, y)
            cav = clf.coef_[0]
            cav = cav / np.linalg.norm(cav)
            mean_pos = np.mean(np.dot(concept_features, cav))
            mean_neg = np.mean(np.dot(random_features, cav))
            if mean_pos < mean_neg:
                cav = -cav
            return cav 

        def get_mixed8_gradients(img, target_class_idx):
            inception_features.clear()
            gradients.clear()
            tensor = inception_preproc(img).unsqueeze(0).to(self.device)
            tensor.requires_grad_(True)
            output = inception_model(tensor)
            target_logit = output[:, target_class_idx]
            inception_model.zero_grad()
            target_logit.backward()
            grad = gradients['mixed_8'].detach().cpu().numpy().squeeze()
            grad_pooled = grad.mean(axis=(1,2))
            return grad_pooled

        def compute_tcav_score_for_class(class_images, cav, target_class_idx):
            pos = 0
            for img in class_images:
                grad_pooled = get_mixed8_gradients(img, target_class_idx)
                grad_pooled = grad_pooled / np.linalg.norm(grad_pooled)
                if np.dot(grad_pooled, cav) > 0:
                    pos += 1
            return pos / len(class_images)

        def load_test_images_by_class(test_dir):
            class_images = {}
            for class_label in os.listdir(test_dir):
                if class_label.startswith('.'):
                    continue
                class_path = os.path.join(test_dir, class_label)
                if not os.path.isdir(class_path):
                    continue
                imgs = []
                for fname in os.listdir(class_path):
                    if fname.lower().endswith(('png', 'jpg', 'jpeg')):
                        img = Image.open(os.path.join(class_path, fname)).convert('RGB')
                        imgs.append(img)
                class_images[class_label] = imgs
            return class_images

        def plot_tcav_heatmap(tcav_matrix, class_labels, cluster_labels, client_id, out_dir="cluster_visualizations", concept_class="???"):
            """
            tcav_matrix: np.ndarray of shape (num_clusters, num_classes)
            class_labels: list of class names (columns)
            cluster_labels: list of cluster names/ids (rows)
            client_id: client numeric id
            out_dir: directory to save the output image
            """
            import matplotlib
            import matplotlib.pyplot as plt
            os.makedirs(out_dir, exist_ok=True)
            fig, ax = plt.subplots(figsize=(1.2 * len(class_labels) + 2, 0.8 * len(cluster_labels) + 3))
            cmap = matplotlib.colors.LinearSegmentedColormap.from_list('red_green', ['red', 'yellow', 'green'])
            cax = ax.imshow(tcav_matrix, cmap=cmap, vmin=0., vmax=1.)

            for i in range(tcav_matrix.shape[0]):
                for j in range(tcav_matrix.shape[1]):
                    val = tcav_matrix[i, j]
                    ax.text(j, i, f"{val:.2f}", va='center', ha='center',
                            color="black" if val < 0.5 else "white", fontsize=10, fontweight='bold')
            ax.set_xticks(np.arange(len(class_labels)))
            ax.set_yticks(np.arange(len(cluster_labels)))
            ax.set_xticklabels(class_labels, rotation=45, ha="right", fontsize=12)
            ax.set_yticklabels(cluster_labels, fontsize=12)
            ax.set_xlabel("Class label")
            ax.set_ylabel("Concept (Cluster)")
            plt.title(f"TCAV class {concept_class} Concepts Contribution\nClient {client_id}", fontsize=15, pad=25)
            plt.colorbar(cax, ax=ax, fraction=0.045)
            plt.tight_layout()
            plt.savefig(os.path.join(out_dir, f"TCAV_heatmap_class_{concept_class}_client{client_id}.png"))
            plt.close(fig)

        final_clusters = []
        final_info = []
        per_class_clusters = defaultdict(lambda: defaultdict(list))
        for meta in self.metas:
            class_label = meta['original_label']
            cluster_id = meta['cluster_id']
            per_class_clusters[class_label][cluster_id].append(meta)

        for class_label, cluster_dict in per_class_clusters.items():
            metas_class = []
            for cluster in cluster_dict.values():
                metas_class.extend(cluster)
            num_discovery_images = len(set(m['original_image_index'] for m in metas_class))
            print(f"{num_discovery_images} unique discovery images for class {class_label}")

            for c_id, cluster in cluster_dict.items():
                img_indices = [m['original_image_index'] for m in cluster]
                unique_imgs = set(img_indices)
                cluster_size = len(cluster)
                num_unique_imgs = len(unique_imgs)
                clip_scores = [m['clip_score'] for m in cluster]
                mean_clip_score = np.mean(clip_scores) if clip_scores else 0
                keep = num_unique_imgs >= 10 or (cluster_size >= 30 and mean_clip_score > 0.75)
                print(
            f"Class {class_label} Cluster {c_id}: "
            f"size={cluster_size}, unique_imgs={num_unique_imgs}, "
            f"mean_clip_score={mean_clip_score:.3f}, keep={keep}")
                for meta in cluster:
                    meta["cluster_id"] = c_id
                final_clusters.append(cluster)
                final_info.append({
                "class_label": class_label,
                "cluster_id": c_id,
                "num_members": cluster_size,
                "num_images": num_unique_imgs,
                "mean_clip_score": mean_clip_score
                })
        
        clusters_by_class = self.build_clusters_by_class(final_clusters,final_info)
        for class_label, clusters in clusters_by_class.items():
            for cid, (cluster_metas, info) in enumerate(clusters):
                self.plot_cluster_4x4(cluster_metas, cid, class_label)
        self.visualize_clusters_with_overlay_and_arrow(clusters_by_class, max_per_cluster=4)
        # 3D UMAP Visualization per class
        self.visualize_clusters_umap_3d(clusters_by_class)

        # Construct cluster CAVs
        cluster_cavs = {}
        cav_results = {}    
        for class_label, clusters in clusters_by_class.items():
            for cluster, info in clusters:
                cid = info['cluster_id']
                other_same_class = [m for c, i in clusters if i['cluster_id'] != cid for m in c]
                other_classes = [m for l, cls in clusters_by_class.items() if l != class_label for c, _ in cls for m in c]
                n_pos = len(cluster)
                n_same = min(n_pos // 2, len(other_same_class))
                n_other = min(n_pos - n_same, len(other_classes))
                if n_same + n_other == 0:
                    continue
                neg_samples = random.sample(other_same_class, n_same) + random.sample(other_classes, n_other)
                pos_feats = extract_mixed8_features(cluster)
                neg_feats = extract_mixed8_features(neg_samples)
                cav = calculate_cav(pos_feats, neg_feats)
                cluster_cavs[(class_label, cid)] = cav
                cav_results[f"{class_label}_{cid}"] = {
                    "cav": cav.tolist(),
                    "count": len(cluster)}
                metrics["cav_results"] = json.dumps(cav_results)
            print(f"all the related CAV of Class {class_label} was successfully generated : \n {cluster_cavs} \n")
          
        test_dir = '/gpfs/helios/home/mahmouds/Thesis/Test'
        image_dir= '/gpfs/helios/home/mahmouds/Thesis/data/ILSVRC2012/segmentation'
        class_to_inception_idx = {
            'basketball': 816,
            #'corn ears': 499,
            #'electric ray': 445,
            'mountain bike': 162,
            'moving van': 191
            #'soccer ball': 817
        }
        test_images = load_test_images_by_class(test_dir)

        #for (class_label, cid), cav in cluster_cavs.items():
        #    if class_label not in test_images or class_label not in class_to_inception_idx:
        #        continue
        #    score = compute_tcav_score_for_class(test_images[class_label], cav, class_to_inception_idx[class_label])
        #    print(f" [Client {self.cid}] TCAV score for class {class_label} cluster {cid}: {score:.2f}")
        #    #### ---- I need to report properly the test image with the contributed clusters ------####
        # ----- Build list of all clusters and class labels for the matrix dimensions
        clusters = list(cluster_cavs.keys())
        cluster_labels = [f"{self.label_to_classname.get(cl, cl)}-C{cid}" for cl, cid in clusters]
        class_labels = sorted(test_images.keys())
        unique_classes = sorted(set(cl for cl, _ in clusters))
        # Initialize the matrix
        tcav_matrix = np.zeros((len(clusters), len(class_labels)))

# Fill matrix: rows=clusters, cols=class labels
        for i, (class_label, cid) in enumerate(clusters):
            for j, test_class in enumerate(class_labels):
                if test_class not in test_images or class_label not in class_to_inception_idx:
                    tcav_matrix[i, j] = np.nan  # gray if missing
                    continue
                score = compute_tcav_score_for_class(
                    test_images[test_class],
                    cluster_cavs[(class_label, cid)],
                    class_to_inception_idx[test_class]
                )
                tcav_matrix[i, j] = score
        for concept_class in unique_classes:
            rows = [i for i, (cl, _) in enumerate(clusters) if cl == concept_class]
            if not rows:
                continue  # No clusters for this class, skip
            submatrix = tcav_matrix[rows, :]
            sub_cluster_labels = [cluster_labels[i] for i in rows]
            
        # After the loop, immediately call:
            #plot_tcav_heatmap(tcav_matrix, class_labels, cluster_labels, client_id=self.cid)
            plot_tcav_heatmap(submatrix, class_labels, sub_cluster_labels, client_id=self.cid, out_dir="cluster_visualizations",concept_class=concept_class)
            print(f"[Client {self.cid}] Saved TCAV class{concept_class} concepts heatmap to cluster_visualizations/TCAV_heatmap_client{self.cid}.png")
        dummy_array = np.zeros((self.k, self.features.shape[1]), dtype=np.float32)
        #print(f"this is the metrics after the local cav: {metrics}")
        return cav_results

### Main funtion

In [7]:
!export FLWR_LOG_LEVEL=DEBUG
!export GRPC_VERBOSITY=DEBUG

/gpfs/space/software/cluster_software/spack/linux-rhel9-x86_64/gcc-9.2.0/python-3.12.3-jfsvfhxwrihwyloazioovxwqtuwcg6qf/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=2585387) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [8]:
def precompute_client_data(num_clients=3):
    import pickle
    import logging
    #logging.getLogger('root').setLevel(logging.WARNING)
    root_dir = '/gpfs/helios/home/mahmouds/Thesis/data/ILSVRC2012/segmentation'
    SAM_transform = transforms.Compose([transforms.Resize((224, 224))])
    dino_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    imagenet_dataset = datasets.ImageFolder(root=root_dir, transform=SAM_transform)
    
   

        # Load SAM2 model
    sam2_checkpoint = "/gpfs/helios/home/mahmouds/sam2/checkpoints/sam2.1_hiera_large.pt"
    model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

    assert os.path.exists(sam2_checkpoint), f"SAM2 checkpoint {sam2_checkpoint} not found."

    sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device)
    dinov2_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device).eval() #ViT-B/32
    mask_gen = SAM2AutomaticMaskGenerator(
            model=sam2,
            predicted_iou_thresh=0.88,
            stability_score_thresh=0.8,
            box_nms_thresh=0.5,
            crop_n_layers=0,
            min_mask_region_area=256,
            max_num_masks=30
        )

        # Dataset and segmentation
    SAM_transform = transforms.Compose([transforms.Resize((224, 224))])
    imagenet_dataset = datasets.ImageFolder(root=root_dir, transform=SAM_transform)
    for cid in range(num_clients):
        print(f"🔧 Starting SAM2 segmentation and saving segments with metadata for client {cid} ...")
        #logger.info(f"Precomputing for client {cid}...")
        segment_root = f'segment_dataset/segments/segments_client_{cid}'
        os.makedirs(segment_root, exist_ok=True)
        #client_features, client_labels, client_metas = [], [], []
        #for img_idx, (image_tensor, label) in tqdm(enumerate(imagenet_dataset), total=len(imagenet_dataset)):
        #    if img_idx % num_clients != cid:
        #        continue

        #    orig_img_path, _ = imagenet_dataset.samples[img_idx]
        #    img_pil = Image.open(orig_img_path).convert("RGB")
        #    img_np = np.array(img_pil)
            
        #    try:
        #        masks = mask_gen.generate(img_np)
        #        if not masks or len(masks) == 0:
        #            print(f"[Client {cid}] ⚠️ No masks returned for image {img_idx}: {orig_img_path}")
        #            continue    
        #    except Exception as e:
        #        print(f"[Client {cid}] Segmentation failed on image {orig_img_path}: {e}")
        #        continue

        #    segments, scores, bboxes = [], [], []
        #    for m in masks:
        #        mask = m["segmentation"].astype(bool)
        #        seg = img_np.copy()
        #        seg[~mask] = 0
        #        segments.append(seg)
        #        scores.append(m["predicted_iou"])
        #        bboxes.append(get_bbox_from_mask(mask))

        #    class_name = imagenet_dataset.classes[label]
        #    class_dir = os.path.join(segment_root, class_name)

        #    save_segments_with_metadata(
        #        img_idx, label, segments, scores, bboxes, orig_img_path, class_dir, score_threshold=0.65
        #    )

        print(f"[Client {cid}] Segmentation done. Starting feature extraction...")
            
        
        #segment_dataset = SegmentDataset(segment_root=segment_root, transform=dino_transform)
        #print(segment_dataset.get_stats())
        #segment_loader = DataLoader(segment_dataset, batch_size=512, shuffle=False, collate_fn=custom_collate)
        
        #features, labels, metas = extract_features_dinov2(segment_loader, dinov2_model, device)
        #print(f"[Client {cid}] Extracted features: {features.shape}, Labels: {len(labels)}")
        #data = {'features': features, 'labels': labels, 'metas': metas}
        #with open(f'segment_dataset/segments/precomputed_client_{cid}.pkl', 'wb') as f:
        #        pickle.dump(data, f)
        logging.info(f"Precomputed data saved for client {cid}.")
    
if __name__ == "__main__":
    dinov2_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device).eval()
    number_of_clusters =15
    num_clients = 3
    np.random.seed(42)

    root_dir_init = '/gpfs/helios/home/mahmouds/Thesis/root/'
    def dino_transform():
        return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    transform_init = transforms.Compose([transforms.Resize((224, 224))])
    dataset_init = datasets.ImageFolder(root=root_dir_init, transform=transform_init)
    dino_transform = dino_transform()
    
    class_to_img_path = defaultdict(list)
    for img_idx, (img_tensor, label) in enumerate(dataset_init):
        img_path, _ = dataset_init.samples[img_idx]
        class_to_img_path[label].append(img_path)
    
    initial_centroids_per_class = {}
    sampled_imgs = []
    class_labels = []
     #Compute all DINOv2 features per class
    initial_centroids_per_class = {}
    for class_label, paths in class_to_img_path.items():
        img_tensors = []
        for img_path in paths:
            img_pil = Image.open(img_path).convert("RGB")
            img_tensors.append(dino_transform(img_pil))
        batch = torch.stack(img_tensors).to(device)
        with torch.no_grad():
            features = dinov2_model(batch).cpu().numpy()
        # Get KMeans++ initial centroids for this class
        kmeans = KMeans(n_clusters=number_of_clusters, init='k-means++', random_state=42)
        kmeans.fit(features)
        initial_centroids_per_class[class_label] = kmeans.cluster_centers_  # shape: (K, D)
        torch.cuda.empty_cache()
        print(f"Computed {number_of_clusters} centroids for class {class_label}")

    print(f"\nTotal classes initialized: {len(initial_centroids_per_class)}")
    strategy = FedKMeansStrategy(initial_centroids=initial_centroids_per_class, num_clusters=number_of_clusters, tol=.5)
    precompute_client_data(num_clients)
    
    def client_fn(context):
     import pickle
     cid = int(context.node_config['partition-id'])
     print(f"[Client {cid}] Starting initialization...")

     try:
        print(f"[Client {cid}] Attempting to load pickle file...") 
        with open(f'segment_dataset/segments/precomputed_client_{cid}.pkl', 'rb') as f:
            data = pickle.load(f)
        print(f"[Client {cid}] Pickle file loaded successfully")
        features = data['features']
        labels = data['labels']
        metas = data['metas']
        print(f"[Client {cid}] Data extracted: features={features.shape}, labels={len(labels)}, metas={len(metas)}")
        client = ThesisKMeansClient(cid,features, labels, metas, device)
        print(f"[Client {cid}] ThesisKMeansClient created successfully")
        return client.to_client()
     except Exception as e:
        print(f"[Client {self.cid}] failed: {e}")
        raise e

    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=50),
        strategy=strategy,
        client_resources={"num_cpus": 4, "num_gpus": 0.25},
        ray_init_args={"object_store_memory": 5 * 1024 * 1024 * 1024,  # 5GiB
            "runtime_env": {"pip": [
    "torch", "torchvision", "flwr", "xformers", "umap-learn", 
            "kneed", "scikit-learn", "opencv-python-headless", "clip", 
            "ftfy", "regex", "tqdm", "Pillow"
]}}
    )


Using cache found in /gpfs/helios/home/mahmouds/.cache/torch/hub/facebookresearch_dinov2_main
/gpfs/helios/home/mahmouds/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/gpfs/helios/home/mahmouds/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/gpfs/helios/home/mahmouds/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


Computed 15 centroids for class 0
Computed 15 centroids for class 1
Computed 15 centroids for class 2

Total classes initialized: 3


Using cache found in /gpfs/helios/home/mahmouds/.cache/torch/hub/facebookresearch_dinov2_main
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=50, no round_timeout


🔧 Starting SAM2 segmentation and saving segments with metadata for client 0 ...
[Client 0] Segmentation done. Starting feature extraction...
🔧 Starting SAM2 segmentation and saving segments with metadata for client 1 ...
[Client 1] Segmentation done. Starting feature extraction...
🔧 Starting SAM2 segmentation and saving segments with metadata for client 2 ...
[Client 2] Segmentation done. Starting feature extraction...


2025-08-16 22:22:19,963	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.16.10.124': 1.0, 'accelerator_type:V100': 1.0, 'CPU': 64.0, 'object_store_memory': 5368709120.0, 'memory': 440261864448.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 0.25}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 4 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 0.0, {}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


...

INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 49]
INFO :      configure_fit: no clients selected, cancel
INFO :      fit progress: (49, 0.0, {}, 2046.5681591406465)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)


[Server] Simulation finished after global TCAV. No more rounds.
[Server] Global evaluation for round 49 skipped (no global model).


(ClientAppActor pid=2589915) Using cache found in /gpfs/helios/home/mahmouds/.cache/torch/hub/facebookresearch_dinov2_main [repeated 6x across cluster]
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 50]
INFO :      configure_fit: no clients selected, cancel
INFO :      fit progress: (50, 0.0, {}, 2050.4403472691774)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)


[Server] Simulation finished after global TCAV. No more rounds.
[Server] Global evaluation for round 50 skipped (no global model).
(ClientAppActor pid=2589914) [Client 0] ThesisKMeansClient created successfully [repeated 6x across cluster]
(ClientAppActor pid=2589914) [Client 0] evaluate() called. [repeated 6x across cluster]
(ClientAppActor pid=2589915) [Client 2] Starting initialization... [repeated 6x across cluster]
(ClientAppActor pid=2589915) [Client 2] Attempting to load pickle file... [repeated 6x across cluster]
(ClientAppActor pid=2589915) [Client 2] Pickle file loaded successfully [repeated 6x across cluster]
(ClientAppActor pid=2589915) [Client 2] Data extracted: features=torch.Size([2088, 384]), labels=2088, metas=2088 [repeated 6x across cluster]
(ClientAppActor pid=2589915) [Client] 2 Initializing with features: <class 'torch.Tensor'>, shape: torch.Size([2088, 384]) [repeated 6x across cluster]
(ClientAppActor pid=2589915) [Client] 2 Labels: <class 'torch.Tensor'>, lengt

INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 50 round(s) in 2054.44s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.0
INFO :      		round 2: 0.0
INFO :      		round 3: 0.0
INFO :      		round 4: 0.0
INFO :      		round 5: 0.0
INFO :      		round 6: 0.0
INFO :      		round 7: 0.0
INFO :      		round 8: 0.0
INFO :      		round 9: 0.0
INFO :      		round 10: 0.0
INFO :      		round 11: 0.0
INFO :      		round 12: 0.0
INFO :      		round 13: 0.0
INFO :      		round 14: 0.0
INFO :      		round 15: 0.0
INFO :      		round 16: 0.0
INFO :      		round 17: 0.0
INFO :      		round 18: 0.0
INFO :      		round 19: 0.0
INFO :      		round 20: 0.0
INFO :      		round 21: 0.0
INFO :      		round 22: 0.0
INFO :      		round 23: 0.0
INFO :      		round 24: 0.0
INFO :      		round 25: 0.0
INFO :      		round 26: 0.0
INFO :      		round 27: 0.0
INFO :      		round 28: 0.0
INFO :      		round 29: 0.0
INFO 